In [1]:
import pandas as pd

In [2]:
cosmetic = pd.read_csv("/content/cosmetics.csv")
food = pd.read_csv("/content/ingredients v1.csv")

In [3]:
cosmetic_ingredients = cosmetic["Ingredients"]
food_ingredients = food["features.value"]

In [4]:
cosmetic_ingredients

,Ingredients
0,"Algae (Seaweed) Extract, Mineral Oil, Petrolat..."
1,"Galactomyces Ferment Filtrate (Pitera), Butyle..."
2,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary..."
3,"Algae (Seaweed) Extract, Cyclopentasiloxane, P..."
4,"Water, Snail Secretion Filtrate, Phenyl Trimet..."
...,...
1467,"Water, Alcohol Denat., Potassium Cetyl Phospha..."
1468,"Water, Isododecane, Dimethicone, Butyloctyl Sa..."
1469,"Water, Dihydroxyacetone, Glycerin, Sclerocarya..."
1470,"Water, Dihydroxyacetone, Propylene Glycol, PPG..."


In [5]:
food_ingredients

,features.value
0,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin."
1,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,..."
2,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ..."
3,Mechanically hulled seasame seeds.Allergy Info...
4,FALSE
...,...
3981,"Unbleached Wheat Flour,Sugar,Palm Fruit Oil,Gr..."
3982,"Wheat Flour,Niacin,Iron,Thiamin Mononitrate,Ri..."
3983,"Enriched Wheat Flour,Semisweet Chocolate Chips..."
3984,"Sodium Bicarbonate,Calcium Phosphate,Niacin,Th..."


In [6]:
cosmetic_ingredients = cosmetic_ingredients.str.split(',')

In [7]:
cosmetic_ingredients

,Ingredients
0,"[Algae (Seaweed) Extract, Mineral Oil, Petro..."
1,"[Galactomyces Ferment Filtrate (Pitera), Buty..."
2,"[Water, Dicaprylyl Carbonate, Glycerin, Cet..."
3,"[Algae (Seaweed) Extract, Cyclopentasiloxane,..."
4,"[Water, Snail Secretion Filtrate, Phenyl Tri..."
...,...
1467,"[Water, Alcohol Denat., Potassium Cetyl Phos..."
1468,"[Water, Isododecane, Dimethicone, Butylocty..."
1469,"[Water, Dihydroxyacetone, Glycerin, Scleroc..."
1470,"[Water, Dihydroxyacetone, Propylene Glycol, ..."


In [8]:
# Step 2: Preprocess each ingredient
def clean_ingredient(ingredient):
    # Convert to lowercase
    ingredient = ingredient.lower()
    # Remove special characters and numbers
    ingredient = ''.join(char for char in ingredient if char.isalpha() or char.isspace())
    # Strip extra whitespace
    ingredient = ingredient.strip()
    return ingredient

# Apply cleaning function to all ingredients
cosmetic_ingredients = cosmetic_ingredients.apply(lambda x: [clean_ingredient(i) for i in x])

# Step 3: Flatten the data (Optional)
# Create a new DataFrame where each row is a single ingredient
ingredients_flat = pd.DataFrame({
    "Ingredient": [ingredient for sublist in cosmetic_ingredients for ingredient in sublist]
})

# Drop duplicates and clean empty strings
ingredients_flat = ingredients_flat.drop_duplicates()
ingredients_flat_cosmetics = ingredients_flat[ingredients_flat['Ingredient'] != ""]

# Save cleaned and flattened data
ingredients_flat_cosmetics.to_csv("cleaned_ingredients_cosmetic.csv", index=False)
print("Cleaned dataset saved!")

Cleaned dataset saved!


In [13]:
food_ingredients = food_ingredients.str.split(',')

In [14]:
# Step 2: Preprocess each ingredient
def clean_ingredient(ingredient):
    # Convert to lowercase
    if isinstance(ingredient, str): # Check if the ingredient is a string
        ingredient = ingredient.lower()
        # Remove special characters and numbers
        ingredient = ''.join(char for char in ingredient if char.isalpha() or char.isspace())
        # Strip extra whitespace
        ingredient = ingredient.strip()
    # Handle non-string values (e.g., replace with empty string)
    else:
        ingredient = "" # or any other suitable handling
    return ingredient

# Apply cleaning function to all ingredients
food_ingredients = food_ingredients.apply(lambda x: [clean_ingredient(i) for i in x] if isinstance(x, list) else []) # Apply cleaning only if x is a list

# Step 3: Flatten the data (Optional)
# Create a new DataFrame where each row is a single ingredient
ingredients_flat_food = pd.DataFrame({
    "Ingredient": [ingredient for sublist in food_ingredients for ingredient in sublist]
})

# Drop duplicates and clean empty strings
ingredients_flat_food = ingredients_flat_food.drop_duplicates()
ingredients_flat_food = ingredients_flat_food[ingredients_flat_food['Ingredient'] != ""] # Use the correct DataFrame name: ingredients_flat_food

# Save cleaned and flattened data
ingredients_flat_food.to_csv("cleaned_ingredients_food.csv", index=False)
print("Cleaned dataset saved!")

Cleaned dataset saved!


In [15]:
ingredients_flat_food

,Ingredient
0,dried prunes
1,water
2,corn syrup
3,sugar
4,pectin
...,...
58090,flour salt
58098,natural cinnamon essence
58121,semisweet chocolate chips
58164,quality black tea and natural blueberry flavors


In [16]:
cosmetics_file = "cleaned_ingredients_cosmetic.csv"  # Replace with your file path
food_file = "cleaned_ingredients_food.csv"  # Replace with your file path

In [17]:
cosmetics_df = pd.read_csv(cosmetics_file, header=None, names=["Ingredient"])
food_df = pd.read_csv(food_file, header=None, names=["Ingredient"])

In [18]:
# Add labels: 1 for cosmetics, 0 for food
cosmetics_df['Label'] = 1
food_df['Label'] = 0

In [19]:
# Cleaning function
def clean_ingredient(ingredient):
    ingredient = ingredient.lower()  # Convert to lowercase
    ingredient = ''.join(char for char in ingredient if char.isalpha() or char.isspace())  # Remove special characters
    return ingredient.strip()  # Strip whitespace

In [20]:
# Clean the ingredients
cosmetics_df['Ingredient'] = cosmetics_df['Ingredient'].apply(clean_ingredient)
food_df['Ingredient'] = food_df['Ingredient'].apply(clean_ingredient)

In [21]:
# Combine the datasets
combined_df = pd.concat([cosmetics_df, food_df], ignore_index=True)

# Shuffle the dataset (optional, for better model training)
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [22]:
# Save the processed dataset
combined_df.to_csv("processed_ingredients_dataset.csv", index=False)
print("Processed dataset saved successfully!")

# Preview the dataset
print(combined_df.head())

Processed dataset saved successfully!
                                        Ingredient  Label
0                    lemon juice concentrate water      0
1                       organic coconut palm sugar      0
2         white grape juice from concentrate water      0
3  natural flavors including natural coffee flavor      0
4                            peg glyceryl stearate      1


In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch





import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from imblearn.over_sampling import SMOTE

# Load the processed dataset
file_path = "processed_ingredients_dataset.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Separate features (X) and labels (y)
X = df['Ingredient']
y = df['Label']

# Convert text data into numerical format using tokenization
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
X_encoded = tokenizer(list(X), truncation=True, padding=True, max_length=128, return_tensors="np")["input_ids"]

# Use SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_encoded, y)

# Convert resampled data back to a DataFrame
X_resampled_texts = tokenizer.batch_decode(X_resampled, skip_special_tokens=True)  # Decode back to text
resampled_df = pd.DataFrame({
    "Ingredient": X_resampled_texts,
    "Label": y_resampled
})

# Shuffle the dataset (optional, but useful for balanced training)
resampled_df = resampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the resampled dataset
resampled_df.to_csv("balanced_ingredients_dataset.csv", index=False)
print("Balanced dataset saved successfully!")
print(resampled_df['Label'].value_counts())  # Check the class distribution


# Load the processed dataset
file_path = "balanced_ingredients_dataset.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Ingredient'], df['Label'], test_size=0.2, random_state=42
)

# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)



Balanced dataset saved successfully!
Label
0    10844
1    10844
Name: count, dtype: int64


In [46]:
# Create PyTorch datasets
class IngredientsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = IngredientsDataset(train_encodings, list(train_labels))
test_dataset = IngredientsDataset(test_encodings, list(test_labels))

In [47]:
# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bert_results",        # Output directory
    num_train_epochs=4,                 # Number of epochs
    per_device_train_batch_size=16,     # Batch size for training
    per_device_eval_batch_size=16,      # Batch size for evaluation
    warmup_steps=500,                   # Number of warmup steps
    weight_decay=0.01,                  # Weight decay
    logging_dir="./bert_logs",          # Directory for logs
    logging_steps=10,
    evaluation_strategy="epoch",        # Evaluate at the end of each epoch
    save_strategy="epoch",              # Save model at each epoch
    load_best_model_at_end=True         # Load the best model at the end of training
)

# Define a Trainer
trainer = Trainer(
    model=model,                         # The pre-trained BERT model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset            # Testing dataset
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.245700,0.199223
2,0.182200,0.187371
3,0.034300,0.201321
4,0.171100,0.214406


Epoch,Training Loss,Validation Loss
1,0.245700,0.199223
2,0.182200,0.187371
3,0.034300,0.201321
4,0.171100,0.214406


TrainOutput(global_step=4340, training_loss=0.13598194268107208, metrics={'train_runtime': 1680.7261, 'train_samples_per_second': 41.292, 'train_steps_per_second': 2.582, 'total_flos': 4564976810496000.0, 'train_loss': 0.13598194268107208, 'epoch': 4.0})

In [48]:
# Evaluate the model
from sklearn.metrics import classification_report

# Predict on test data
predictions = trainer.predict(test_dataset)
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

# Print classification report
print(classification_report(test_labels, predicted_labels, target_names=["Non-Cosmetic", "Cosmetic"]))

              precision    recall  f1-score   support

Non-Cosmetic       0.94      0.97      0.95      2173
    Cosmetic       0.96      0.94      0.95      2165

    accuracy                           0.95      4338
   macro avg       0.95      0.95      0.95      4338
weighted avg       0.95      0.95      0.95      4338



In [49]:

# Save the fine-tuned BERT model and tokenizer
model.save_pretrained("fine_tuned_bert")
tokenizer.save_pretrained("fine_tuned_bert")
print("Fine-tuned BERT model and tokenizer saved successfully!")


Fine-tuned BERT model and tokenizer saved successfully!


In [50]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer

model = BertForSequenceClassification.from_pretrained("fine_tuned_bert")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_bert")


In [51]:
from transformers import DistilBertForSequenceClassification, AutoTokenizer
import torch

# Load the fine-tuned model and tokenizer
model_path = "fine_tuned_bert"  # Path to your saved model
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [52]:
def predict_ingredient(input_text):
    # Preprocess the input text
    inputs = tokenizer(input_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Perform prediction
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=1).item()

    # Map label to category
    label_map = {0: "Non-Cosmetic", 1: "Cosmetic"}
    return label_map[predicted_label], probs.detach().numpy()


In [56]:
# Test samples
samples = [
    "Aqua",
    "Papaya Extract",
    "Sodium Laureth Sulphate",
    "Vitamin E",
    "Coco Betaine",
    "Sodium Cocoyl Isethionate",
    "Sodium Benzoate",
    "EDTA",
    "Fragrance",
]

# Predict for each sample
for sample in samples:
    category, probabilities = predict_ingredient(sample)
    print(f"Ingredient: {sample}")
    print(f"Predicted Category: {category}")
    print(f"Probabilities: {probabilities}\n")


Ingredient: Aqua
Predicted Category: Cosmetic
Probabilities: [[0.0294203  0.97057974]]

Ingredient: Papaya Extract
Predicted Category: Non-Cosmetic
Probabilities: [[0.9556189  0.04438116]]

Ingredient: Sodium Laureth Sulphate
Predicted Category: Cosmetic
Probabilities: [[0.01451658 0.9854834 ]]

Ingredient: Vitamin E
Predicted Category: Non-Cosmetic
Probabilities: [[0.9629238  0.03707615]]

Ingredient: Coco Betaine
Predicted Category: Cosmetic
Probabilities: [[0.00270503 0.99729496]]

Ingredient: Sodium Cocoyl Isethionate
Predicted Category: Cosmetic
Probabilities: [[0.00269539 0.99730456]]

Ingredient: Sodium Benzoate
Predicted Category: Cosmetic
Probabilities: [[0.04720719 0.9527928 ]]

Ingredient: EDTA
Predicted Category: Non-Cosmetic
Probabilities: [[0.95283544 0.04716452]]

Ingredient: Fragrance
Predicted Category: Cosmetic
Probabilities: [[0.00585793 0.9941421 ]]

